In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from copy import copy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

# Model Selection

In this notebook we train various classifiers on our dataset of retracted and non-retracted papers to determine the best model for predicting whether a paper will be retracted. The classifiers we investigate are:
1. Logistic Regression
2. $k$-Nearest Neighbours
3. Random Forests
3. Support vector classifiers


The KPIs we are interested in are precision, recall, and $F_1$-score. While we are also interested in the accuracy of our classifiers, this measure is less important to us since the retracted and non-retracted classes are massively unbalanced: as we will see, a classifier that simply predicts no retraction for every paper has a 99.7% accuracy!

## Data Preparation

First, we need to prepare our data for the classifiers. To see how our data was collected, take a look at the `example_openAlex_data.ipynb` notebook. Note that we have already split our data into training and testing datasets: the tranining set contains all papers published in 2010-2020 and the testing set contains all paper published in 2021-2022.

In [174]:
data = pd.read_pickle('./Data/OpenAlex/openalex-data-2010-2020-train-06.pkl')
data.dropna

data.head()

,id,title,publication_year,countries_distinct_count,institutions_distinct_count,referenced_works_count,cited_by_count,authors_distinct_count,any_author_has_retraction,min_retracted_author_rank,...,has_10pct_retracted_author,top_percentile_retracted_author,frac_author_repeat_offenders,any_institution_has_retraction,min_retracted_institution_rank,has_1pct_retracted_institution,has_5pct_retracted_institution,has_10pct_retracted_institution,top_percentile_retracted_institution,is_retracted
0,https://openalex.org/W2919115771,Deep learning,2015,2,3,42,55841,3,False,-inf,...,False,0.000000,0.000000,True,313.0,False,False,True,94.538771,False
1,https://openalex.org/W2117692326,Hallmarks of Cancer: The Next Generation,2011,2,4,247,52921,2,True,735.0,...,True,98.229832,0.500000,True,61.5,False,True,True,98.941012,False
2,https://openalex.org/W2145339207,Human-level control through deep reinforcement...,2015,1,2,19,20397,19,False,-inf,...,False,0.000000,0.000000,False,-inf,False,False,False,0.000000,False
3,https://openalex.org/W3004280078,A pneumonia outbreak associated with a new cor...,2020,1,4,14,17439,29,True,2047.5,...,True,95.064512,0.103448,True,9.0,True,True,True,99.859968,False
4,https://openalex.org/W3009912996,SARS-CoV-2 Cell Entry Depends on ACE2 and TMPR...,2020,3,12,59,16517,13,False,-inf,...,False,0.000000,0.000000,True,257.0,False,True,True,95.518992,False


Here, we are removing features that we do not want to train on, such as `title`, `id`, `publication_year` and `cited_by_count`. While it is true that information such as publication year and the number of citations a work receives can be informative once a paper has been published, these features are very uninformative when considering *new* papers. We are also excluding features involving `min_rank` since some values are `-inf`. Then, we are ensuring that the remaining features are all numerical.

In [175]:
reduced_data = data.drop(columns=['id', 'title', 'publication_year', 'cited_by_count', 'min_retracted_author_rank', 'min_retracted_institution_rank'])

for col in reduced_data.columns:
    if reduced_data[col].dtype == bool:
        reduced_data[col] = reduced_data[col].astype(int)

    elif col in ['publication_year', 'countries_distinct_count', 'institutions_distinct_count', 'referenced_works_count', 'cited_by_count', 'authors_distinct_count']:
        reduced_data[col] = reduced_data[col].astype(float)


We apply an 80/20 train-test split, making sure to stratify according to the `is_retracted` column of our data frame. This ensures that the distribution of retracted papers is constant in both datasets.

In [177]:
X_train, X_test = train_test_split(reduced_data.copy(), test_size=0.2, stratify=reduced_data['is_retracted'])

Finally, we standardize each of the truly numerical features in our dataset. Note that we are not standardizing features that were previously Boolean and are applying the same transformations to both the training and test datasets.

In [178]:
for col in X_train.columns:
    if X_train[col].dtype == float:
        mean = X_train[col].mean()
        std = X_train[col].std()
        X_train[col] = (X_train[col] - mean)/std
        X_test[col] = (X_test[col] - mean)/std  

## Key Performance Indicators

Our KPIs are recall, precision, and $F_1$ score. We use `sklearn`'s confusion matrix to calculate these values.

In [179]:
def recall(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)

    return (conf_mat[1][1])/(conf_mat[1][0]+conf_mat[1][1])

def precision(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)

    return (conf_mat[1][1])/(conf_mat[0][1]+conf_mat[1][1])

def f1(y_true, y_pred):
    conf_mat = confusion_matrix(y_true, y_pred)

    return (conf_mat[1][1])/(conf_mat[1][1]+0.5*(conf_mat[0][1]+conf_mat[1][0]))

Throughout this notebook, we build a dictionary of KPIs for the models we test.

In [189]:
kpis = {
    'models' : [],
    'f1' : [],
    'precision' : [],
    'recall' : []
}

# Logistic Regression

Our baseline model will be a simple logistic regression. Since our dataset is large, we are using the `newton-cg` solver. As this solver only supports $\ell_2$ regularization, we first use forward stepwise selection to choose an optimal set of features for our regressions.

In [181]:
subsets = [[]]

features = list(X_train.columns)
features.remove('is_retracted')

for i in range(len(X_train.columns)-1):
    current_features = copy(subsets[i])

    best_accuracy  = 0

    for feature in features:
        model = LogisticRegression(solver='newton-cg')
        test_features = current_features.copy()
        test_features.append(feature)

        X_tt = X_train[test_features]
        model.fit(X_tt, X_train['is_retracted'])

        accuracy = f1(X_train['is_retracted'], model.predict(X_tt))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            new_feature = feature

    current_features.append(new_feature)
    if features != []:
        features.remove(new_feature)
        subsets.append(current_features)

Now, we define a model for each set of features determined in our forward stepwise selection and employ 10-fold cross-validation to choose the best model. Note that we are again using *stratified* cross-validation to maintain the distribution of retracted and unretracted papers within each dataset.

In [182]:
kfold = StratifiedKFold(n_splits = 10, shuffle=True, random_state=123)

f1_scores = np.zeros((len(subsets), 10))

i = 0

for train_index, test_index in kfold.split(X_train, X_train['is_retracted']):
    X_tt = X_train.iloc[train_index]
    X_val = X_train.iloc[test_index]

    # Baseline model

    baseline = np.zeros(len(X_val))

    f1_scores[0, i] = f1(X_val.is_retracted, baseline)

    for j in range(len(subsets)-1):
        subset = subsets[j+1]
        model = LogisticRegression(solver='newton-cg')

        model.fit(X_tt[subset], X_tt.is_retracted.values)
        f1_scores[j+1,i] = f1(X_val.is_retracted.values, model.predict(X_val[subset]))
    i +=1

In [183]:
for i in range(len(subsets)):
    print('Subset', i, 'F1 Score:',np.mean(f1_scores[i]))

Subset 0 F1 Score: 0.0
Subset 1 F1 Score: 0.13871575281488485
Subset 2 F1 Score: 0.1571756727907499
Subset 3 F1 Score: 0.16143383893196123
Subset 4 F1 Score: 0.1612869478650249
Subset 5 F1 Score: 0.16606550264357964
Subset 6 F1 Score: 0.16609934115330635
Subset 7 F1 Score: 0.16302470711908365
Subset 8 F1 Score: 0.16356783171122782
Subset 9 F1 Score: 0.1614814373533083
Subset 10 F1 Score: 0.1630528428241459
Subset 11 F1 Score: 0.16144338283881146
Subset 12 F1 Score: 0.16144338283881146
Subset 13 F1 Score: 0.15274650552289837
Subset 14 F1 Score: 0.14494006623450667
Subset 15 F1 Score: 0.14476462763801543


In [184]:
f1s = np.mean(f1_scores, axis=1)
best_subset = np.argmax(f1s)

best_subset

6

We can see that Subset 6 has the highest $F_1$ score.

In [185]:
subsets[best_subset]

['frac_author_repeat_offenders',
 'top_percentile_retracted_institution',
 'authors_distinct_count',
 'referenced_works_count',
 'has_1pct_retracted_institution',
 'has_10pct_retracted_institution']

From the list of features in our best subset, we can see that the most informative single feature is the proportion of authors on a paper that have been previously retracted, followed by a measure of how many retractions any institution related to the paper has received. This makes sense with our intuition for features that may be correlated with a risk of retraction. However, when we look at the distribution of weights in our final model, the most highly weighted feature is the boolean value of whether any associated instituion has had a retraction in the past.

In [186]:
model = LogisticRegression()
model.fit(X_train[subsets[best_subset]], X_train['is_retracted'])
model.coef_

array([[ 0.47249703,  1.10095506,  0.11768857,  0.06004498,  0.13091543,
        -0.23509377]])

In [190]:
kpis['models'].append('Logistic Regression')
kpis['f1'].append(f1(X_train['is_retracted'], model.predict(X_train[subsets[best_subset]])))
kpis['precision'].append(precision(X_train['is_retracted'], model.predict(X_train[subsets[best_subset]])))
kpis['recall'].append(recall(X_train['is_retracted'], model.predict(X_train[subsets[best_subset]])))

Here we are generating a visualization of the weights for each feature in our model using the `altair` library.

In [193]:
import altair as alt


def blue():
    font = "Arial"
    
    return {
        "config" : {
             "title": {'font': font},
             "axis": {
                  "labelFont": font,
                  "titleFont": font
             },
             'view': {
                 'height': 300,
                 'width': 600
             },
          'mark': {
                'color': '#9fd1fff2',
                'fill': '#9fd1fff2'
            }   
        }
    }

alt.themes.register('blue', blue)
alt.themes.enable('blue')


ThemeRegistry.enable('blue')

In [194]:
feature_weights = pd.DataFrame({'features':subsets[best_subset], 'weights':model.coef_[0]})

In [195]:
base = alt.Chart(feature_weights, title=alt.Title('Feature Weights for Logistic Regression')).encode(
    y=alt.Y('weights').title('Weight'),
    x=alt.X('features', axis=alt.Axis(labelAngle=-45)).title('Feature').sort('-y'),
    text=alt.Text('weights', format='0.2f')
).properties(
    width = 600,
    height = 200
)

base.mark_bar() + base.mark_text(dy=-5)

alt.LayerChart(...)

# $k$-Nearest Neighbors

We build models that classify our data based on 1-10 neighbours and use stratified 10-fold cross validation to determine an optimal number of neighbours to consider in our model.

In [196]:
from sklearn.neighbors import KNeighborsClassifier

In [197]:
features = list(X_train.columns)
features.remove('is_retracted')

kfold = StratifiedKFold(n_splits = 10, shuffle=True, random_state=123)

f1_scores = np.zeros((10, 10))

i = 0

for train_index, test_index in kfold.split(X_train[features], X_train['is_retracted']):
    X_tt = X_train.iloc[train_index]
    X_val = X_train.iloc[test_index]

    for j in range(9):
        model = KNeighborsClassifier(n_neighbors=j+1, n_jobs=10)

        model.fit(X_tt, X_tt.is_retracted.values)
        f1_scores[j,i] = f1(X_val.is_retracted.values, model.predict(X_val))
    i +=1



In [198]:
for i in range(9):
    print(i+1, 'Neighbors F1 Score:',np.mean(f1_scores[i]))

1 Neighbors F1 Score: 0.895280091156291
2 Neighbors F1 Score: 0.8086767338639695
3 Neighbors F1 Score: 0.8291684265803049
4 Neighbors F1 Score: 0.7496091869234942
5 Neighbors F1 Score: 0.774752522886418
6 Neighbors F1 Score: 0.7169432792669954
7 Neighbors F1 Score: 0.7334112358293066
8 Neighbors F1 Score: 0.6799697489271957
9 Neighbors F1 Score: 0.6948059094216114


We can see that using only the first nearest neighbour gives us the best $F_1$ score, however this model will naturally have very high variance. Thus, we choose to use the top 3 nearest neighbours, since this has a high $F_1$ score with smaller variance.

In [199]:
model_3nn = KNeighborsClassifier(n_neighbors=3, n_jobs=10)
model_3nn.fit(X_train[features], X_train['is_retracted'])

kpis['models'].append('3-Nearest Neighbours')
kpis['f1'].append(f1(X_train['is_retracted'], model_3nn.predict(X_train[features])))
kpis['precision'].append(precision(X_train['is_retracted'], model_3nn.predict(X_train[features])))
kpis['recall'].append(recall(X_train['is_retracted'], model_3nn.predict(X_train[features])))

# Support Vector Classification

Here we use a polynomial kernel of degree 3. This was chosen by doing 10-fold cross validation on a much smaller dataset.

In [132]:

features = list(X_train.columns)
features.remove('is_retracted')

model_svc = SVC(kernel='poly', degree = 3, gamma='auto')
model_svc.fit(X_train[features], X_train['is_retracted'])

kpis['models'].append('SVC')
kpis['f1'].append(f1(X_train['is_retracted'], model_svc.predict(X_train[features])))
kpis['precision'].append(precision(X_train['is_retracted'], model_svc.predict(X_train[features])))
kpis['recall'].append(recall(X_train['is_retracted'], model_svc.predict(X_train[features])))


# Random Forest



In [200]:
from sklearn.ensemble import RandomForestClassifier

In [201]:
model = RandomForestClassifier(n_estimators=500, max_depth=20)

model.fit(X_train[features], X_train['is_retracted'])

model.predict(X_test[features])
f1(X_train['is_retracted'], model.predict(X_train[features]))

0.8576544315129812

In [202]:
kpis['models'].append('Random Forest')
kpis['f1'].append(f1(X_train['is_retracted'], model.predict(X_train[features])))
kpis['precision'].append(precision(X_train['is_retracted'], model.predict(X_train[features])))
kpis['recall'].append(recall(X_train['is_retracted'], model.predict(X_train[features])))

# Visualizations

In [203]:
kpis_df = pd.DataFrame(kpis)
kpis_df = kpis_df.melt(id_vars='models', value_vars=['f1','precision', 'recall'], var_name='KPI', value_name='Score')

In [204]:
plot = alt.Chart(kpis_df, title=alt.Title('KPIs for Each Model')).mark_point().encode(
    x = alt.X('models', title='Model'),
    y = alt.Y('Score'),
    color = 'KPI')

plot

alt.Chart(...)

## Test Data

We can see above that our random forest classifier far outperforms all other models in $F_1$ score, precision and recall. We now test our model on papers published in 2021-2022.

In [206]:
model = RandomForestClassifier(n_estimators=500, max_depth=20)
model.fit(X_train[features], X_train['is_retracted'])

RandomForestClassifier(max_depth=20, n_estimators=500)

In [207]:
confusion_matrix(X_test['is_retracted'], model.predict(X_test[features]))

array([[84673,    13],
       [  130,    29]])

In [208]:
print('F1:', f1(X_test['is_retracted'], model.predict(X_test[features])))
print('Precision:', precision(X_test['is_retracted'], model.predict(X_test[features])))
print('Recall:', recall(X_test['is_retracted'], model.predict(X_test[features])))

F1: 0.2885572139303483
Precision: 0.6904761904761905
Recall: 0.18238993710691823
